In [1]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

TRAIN_IMAGE_DIR = './dogs-vs-cats-identification/train' 

filenames = os.listdir(TRAIN_IMAGE_DIR)

df = pd.DataFrame({
    'filename': filenames,
    'class': ['cat' if 'cat' in fname else 'dog' for fname in filenames]
})

train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

IMAGE_SIZE = 96
BATCH_SIZE = 32

train_data = datagen.flow_from_dataframe(
    train_df, TRAIN_IMAGE_DIR, x_col='filename', y_col='class',
    target_size=(IMAGE_SIZE, IMAGE_SIZE), class_mode='binary', batch_size=BATCH_SIZE
)

val_data = datagen.flow_from_dataframe(
    val_df, TRAIN_IMAGE_DIR, x_col='filename', y_col='class',
    target_size=(IMAGE_SIZE, IMAGE_SIZE), class_mode='binary', batch_size=BATCH_SIZE
)

base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

filepath = './models/best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

callbacks = [checkpoint]
epochs = 100

history = model.fit(
    train_data, steps_per_epoch=len(train_data), epochs=epochs,
    validation_data=val_data, callbacks=callbacks
)

test_loss, test_acc = model.evaluate(val_data, verbose=2)
print('\nTest accuracy:', test_acc)
model.save('cat_dog_classifier.h5')

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Epoch 1/100
625/625 [==============================] - ETA: 0s - loss: 0.6816 - accuracy: 0.5960
Epoch 1: val_loss improved from inf to 0.63295, saving model to ./models\best_model.epoch01-loss0.63.hdf5


c:\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 333s 528ms/step - loss: 0.6816 - accuracy: 0.5960 - val_loss: 0.6329 - val_accuracy: 0.6448
Epoch 2/100
625/625 [==============================] - ETA: 0s - loss: 0.6355 - accuracy: 0.6369
Epoch 2: val_loss improved from 0.63295 to 0.61862, saving model to ./models\best_model.epoch02-loss0.62.hdf5
625/625 [==============================] - 172s 275ms/step - loss: 0.6355 - accuracy: 0.6369 - val_loss: 0.6186 - val_accuracy: 0.6576
Epoch 3/100
625/625 [==============================] - ETA: 0s - loss: 0.6277 - accuracy: 0.6482
Epoch 3: val_loss improved from 0.61862 to 0.60772, saving model to ./models\best_model.epoch03-loss0.61.hdf5
625/625 [==============================] - 167s 266ms/step - loss: 0.6277 - accuracy: 0.6482 - val_loss: 0.6077 - val_accuracy: 0.6686
Epoch 4/100
625/625 [==============================] - ETA: 0s - loss: 0.6207 - accuracy: 0.6530
Epoch 4: val_loss improved from 0.60772 to 0.60444, saving model to ./models\best_mo

In [4]:
# Assume `model` is the trained model and `val_data` is the validation dataset
test_loss, test_acc = model.evaluate(val_data, verbose=2)

# Calculate the baseline - predict the most frequent class
baseline = max(train_df['class'].value_counts()) / len(train_df)

# Generate the resume statement
accuracy_multiple = test_acc / baseline
statement = f"Achieved a top-1 accuracy of {test_acc*100:.2f}% ({accuracy_multiple:.2f}x baseline)"
print(statement)


157/157 - 28s - loss: 0.5701 - accuracy: 0.7008 - 28s/epoch - 179ms/step
Achieved a top-1 accuracy of 70.08% (1.40x baseline)
